In [11]:
import torch
from torch import nn

m = nn.Conv2d(256, 128, kernel_size=3, padding=1, groups=1, stride=1)
input = torch.randn(1, 256, 64, 64)
output = m(input)
print(output.shape)

torch.Size([1, 128, 64, 64])


In [3]:
%cd my_framework_1/content_encoder/modules

import torch
from net_module import AntiAliasInterpolation2d

m = AntiAliasInterpolation2d(3, 0.25)
input = torch.randn(1, 3, 256, 256)
output = m(input)
print(output.shape)

[Errno 2] No such file or directory: 'my_framework_1/content_encoder/modules'
/root/TalkingHead/my_framework_1/content_encoder/modules
torch.Size([1, 3, 64, 64])


In [4]:
%cd my_framework_1/content_encoder/modules

import torch
from net_module import Hourglass

m = Hourglass(32, in_features=3, max_features=1024, num_blocks=5)
input = torch.randn(1, 3, 64, 64)
output = m(input)
print(output.shape)

[Errno 2] No such file or directory: 'my_framework_1/content_encoder/modules'
/root/TalkingHead/my_framework_1/content_encoder/modules
torch.Size([1, 35, 64, 64])


In [11]:
%cd my_framework_1/content_encoder/modules

import torch
from net_module import OcclusionAwareGenerator

dense_motion_params = {
    "block_expansion":64,
    "num_blocks": 5,
    "max_features": 1024,
    # "num_kp": 10,
    # "num_channels": 3,
    # "estimate_occlusion_map": True,
    "scale_factor": 0.25
}
#(64,5,1024,10,3,True,0.25,0.01)

m = OcclusionAwareGenerator(3, 10, block_expansion=64, max_features=512, num_down_blocks=2,
                 num_bottleneck_blocks=6, estimate_occlusion_map=True, 
                 dense_motion_params=dense_motion_params, estimate_jacobian=True)
source_image = torch.randn(1, 3, 256, 256)
kp_driving = {
    "heatmap": torch.randn(1, 10, 58, 58),
    "jacobian": torch.randn(1, 10, 2, 2),
    "value": torch.randn(1, 10, 2)
}
kp_source = {
    "heatmap": torch.randn(1, 10, 58, 58),
    "jacobian": torch.randn(1, 10, 2, 2),
    "value": torch.randn(1, 10, 2)
}
output = m(source_image, kp_driving, kp_source)
print(output["mask"].shape)
print(output["sparse_deformed"].shape)
print(output["occlusion_map"].shape)
print(output["deformed"].shape)
print(output["prediction"].shape)

[Errno 2] No such file or directory: 'my_framework_1/content_encoder/modules'
/root/TalkingHead/my_framework_1/content_encoder/modules
torch.Size([1, 11, 64, 64])
torch.Size([1, 11, 3, 64, 64])
torch.Size([1, 1, 64, 64])
torch.Size([1, 3, 256, 256])
torch.Size([1, 3, 256, 256])
